In [ ]:
# -*- coding: utf-8 -*-
!pip install stable-baselines3[extra]

import stable_baselines3
import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from gym import spaces
import numpy as np
import gym
import matplotlib.pyplot as plt

df = pd.read_excel('dock.xlsx')

def clean_data(df):
    for column in df.columns[1:]:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    df = df.dropna()
    return df

df = clean_data(df)

class DockingEnv(gym.Env):
    def __init__(self, df):
        super(DockingEnv, self).__init__()
        self.df = df
        self.current_step = 0
        self.action_space = spaces.Discrete(4)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(4,), dtype=np.float32)

    def reset(self):
        self.current_step = 0
        return self.df.iloc[self.current_step, 1:].values

    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.df)
        reward = self.df.iloc[self.current_step, action + 1] if not done else 0
        obs = self.df.iloc[self.current_step, 1:].values if not done else np.zeros(4)
        return obs, reward, done, {}

    def render(self, mode='human'):
        pass

env = DummyVecEnv([lambda: DockingEnv(df)])

model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)
model.save("docking_rl_model")

obs = env.reset()
actions = []
rewards = []
variants = df.iloc[:, 0].values

for i in range(len(df)):
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    actions.append(action[0])
    rewards.append(reward[0])
    if done:
        break

results_df = pd.DataFrame({
    'Variant': variants[:len(actions)],
    'Action': actions,
    'Reward': rewards
})

drug_names = ['Pimozide', 'Risperidone', 'Aripiprazole', 'Haloperidol']
results_df['Drug'] = results_df['Action'].apply(lambda x: drug_names[x])

best_combinations = results_df.loc[results_df.groupby('Variant')['Reward'].idxmax()]
top_best_combinations = best_combinations.nlargest(20, 'Reward')

print(top_best_combinations)

plt.figure(figsize=(12, 8))
plt.barh(top_best_combinations['Variant'], top_best_combinations['Reward'], color='skyblue')
for index, value in enumerate(top_best_combinations['Reward']):
    plt.text(value, index, f'{value:.2f}', va='center')
plt.xlabel('Reward')
plt.ylabel('Variant')
plt.title('Top 20 Best Drug Combinations by Variant')
plt.show()

drug_summary = top_best_combinations.groupby('Drug').size()
print(drug_summary)
